In [169]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from prophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet
from datetime import datetime

In [57]:
!pip install prophet
     

In [102]:
!pip install fbprophet

  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Using cached cmdstanpy-0.9.5-py3-none-any.whl (37 kB)
  ERROR: Command errored out with exit status 1:
   command: /Users/rayzapien/opt/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/wv/h5pdqs5n7nsg_yqxw3ycpkp40000gp/T/pip-install-rz497ygn/fbprophet_920d27263df14f56baa9516af1a6d7cc/setup.py'"'"'; __file__='"'"'/private/var/folders/wv/h5pdqs5n7nsg_yqxw3ycpkp40000gp/T/pip-install-rz497ygn/fbprophet_920d27263df14f56baa9516af1a6d7cc/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/wv/h5pdqs5n7nsg_yqxw3ycpkp40000gp/T/pip-wheel-t3pa9y5x
       cwd: /private/var/folders/wv/h5pdqs5n7nsg_yqxw3ycpkp40000gp/T/pip-install-rz497ygn/fb

In [157]:
#read in csv for
all_bdrm = pd.read_csv("housing_prices_data.csv")
all_bdrm.dropna(inplace=True)
all_bdrm.head()


,RegionID,Neighborhood,State,City,Metro,County,2020-01-31,2020-02-29,2020-03-31,2020-04-30,...,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,BedroomCount
0,118208,South Los Angeles,CA,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,350403.0,354217.0,357681.0,360467.0,...,485481.0,486580.0,486336.0,486796.0,486585.0,487578.0,486809.0,489862.0,489152.0,1
1,268496,Southeast Los Angeles,CA,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,312659.0,314476.0,315822.0,317131.0,...,410623.0,413743.0,416449.0,419431.0,420848.0,422551.0,422515.0,427628.0,430679.0,1
2,273565,East San Jose,CA,San Jose,"San Jose-Sunnyvale-Santa Clara, CA",Santa Clara County,394852.0,397467.0,399616.0,400423.0,...,465598.0,465629.0,463605.0,460273.0,456309.0,452782.0,449427.0,451437.0,454010.0,1
3,273088,Bullard,CA,Fresno,"Fresno, CA",Fresno County,99098.0,103441.0,107280.0,111859.0,...,154223.0,156623.0,158448.0,159079.0,159166.0,158239.0,157523.0,162523.0,167592.0,1
4,276652,West San Jose,CA,San Jose,"San Jose-Sunnyvale-Santa Clara, CA",Santa Clara County,488197.0,492374.0,495291.0,494556.0,...,552316.0,550898.0,546854.0,542083.0,538433.0,536506.0,535151.0,551396.0,573673.0,1


In [158]:
#transpose data to
#chose the first row in as a test 
transposed_sla = all_bdrm.loc[all_bdrm.index==0]

In [159]:
transposed_sla = transposed_sla.transpose()
transposed_sla


,0
RegionID,118208
Neighborhood,South Los Angeles
State,CA
City,Los Angeles
Metro,"Los Angeles-Long Beach-Anaheim, CA"
County,Los Angeles County
2020-01-31,350403.0
2020-02-29,354217.0
2020-03-31,357681.0
2020-04-30,360467.0


In [160]:
#drop unneeded columns 
transposed_sla.drop(['RegionID','Neighborhood','State','City','Metro','County','BedroomCount'],axis=0, inplace=True)

In [178]:
transposed_sla['month_date'] = transposed_sla.index
transposed_sla

,0,month_date
2020-01-31,350403.0,2020-01-31
2020-02-29,354217.0,2020-02-29
2020-03-31,357681.0,2020-03-31
2020-04-30,360467.0,2020-04-30
2020-05-31,361981.0,2020-05-31
2020-06-30,361653.0,2020-06-30
2020-07-31,362266.0,2020-07-31
2020-08-31,364425.0,2020-08-31
2020-09-30,369158.0,2020-09-30
2020-10-31,374385.0,2020-10-31


In [181]:
#changing the month_date column to datetime data type
transposed_sla['month_date'] = pd.to_datetime(transposed_sla['month_date'], format="%Y-%m-%d")
transposed_sla.head()

,0,month_date
2020-01-31,350403.0,2020-01-31
2020-02-29,354217.0,2020-02-29
2020-03-31,357681.0,2020-03-31
2020-04-30,360467.0,2020-04-30
2020-05-31,361981.0,2020-05-31


In [182]:
transposed_sla

,0,month_date
2020-01-31,350403.0,2020-01-31
2020-02-29,354217.0,2020-02-29
2020-03-31,357681.0,2020-03-31
2020-04-30,360467.0,2020-04-30
2020-05-31,361981.0,2020-05-31
2020-06-30,361653.0,2020-06-30
2020-07-31,362266.0,2020-07-31
2020-08-31,364425.0,2020-08-31
2020-09-30,369158.0,2020-09-30
2020-10-31,374385.0,2020-10-31


In [190]:
#split data into train and test 
train = transposed_sla.iloc[:len(transposed_sla)-24]
test = transposed_sla.iloc[len(transposed_sla)-24:]

In [189]:
type(train)

pandas.core.frame.DataFrame

In [191]:
train.head()

,0,month_date
2020-01-31,350403.0,2020-01-31
2020-02-29,354217.0,2020-02-29
2020-03-31,357681.0,2020-03-31
2020-04-30,360467.0,2020-04-30
2020-05-31,361981.0,2020-05-31


In [ ]:
#train model
# no need to scale data since we are looking at one feeature which is price

In [195]:
model = Prophet()
model

AttributeError: 'Prophet' object has no attribute 'stan_backend'

In [187]:
model.fit(train)

TypeError: fit() missing 1 required positional argument: 'df'